In [20]:
!pip install --upgrade --user --quiet google-cloud-aiplatform
!pip install -qU langchain-google-genai


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import getpass
import os
from dotenv import load_dotenv
import base64
from langchain_core.prompts import HumanMessagePromptTemplate, ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
from IPython.display import Markdown, display
from langchain_google_genai import ChatGoogleGenerativeAI
from vertexai.preview.generative_models import grounding as preview_grounding

In [7]:
load_dotenv()
#os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r""


In [8]:
PROJECT_ID = " "  # @param {type:"string"}
REGION = ""  # @param {type: "string"}
import vertexai
vertexai.init(project=PROJECT_ID, location=REGION)

In [9]:
import vertexai
vertexai.init(project=PROJECT_ID, location=REGION)

In [10]:
model_ground = GenerativeModel("gemini-1.5-flash-001")

NameError: name 'GenerativeModel' is not defined

In [11]:
from vertexai.generative_models import (
    GenerationResponse,
    GenerativeModel,
    Tool,
    grounding,
)


In [12]:
product = "parle g biscuit" 
PROMPT = f"only give me the nutritional(benefits/harms) content regarding the consumptions of {product}. ALso if there is any recent news regarding {product}"
tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())
response = model_ground.generate_content(PROMPT, tools=[tool])
#print(response.candidates[0].content.parts[0].text)
response_grounding = response.candidates[0].content.parts[0].text

NameError: name 'model_ground' is not defined

In [13]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-001",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    
)

In [14]:
chat_prompt = ChatPromptTemplate.from_messages(
    messages=[
        SystemMessage(
            content='''You are well experienced dietician and nutritionist. You will be given the Gae, height, weight and dietary precautions and goals of the person. 
                        You will be given a product and its ingredients select one from each category based on the ingredients and health goals of the person.
                        Category 1: how frequently can i consume it based on my weight goals:
                        1. daily consumption
                        2. weekly consumption
                        3. monthly consumption
                        Category 2: and as per my diet is this food :
                        1. nutritional
                        2. recreations
                        3.  or regular consumption for me

                        Category 3: If the food product follows any of the following:
                        1. Organic
                        2. Sustainable: 
                        3. Supports Small Businesses
                        4. Animal Cruelty-Free: 

                        Category 4: flag alergens if they are related to my allergy otherwise don't flag a warning
                        Category 5: you need to also flag diet type match this with the input of user info and flag if it is suitable for their dietary type or restrictions.
                        Category 6: Alert if there is a higher presence of nutrients desired in low qty (fats, sugar, sodium, calories)
                        Your final answer should only contain tags among one option of each category based on product nutrition analysis.
                        If the answer is None just drom that category output.
                        Finally if the 4 out of 6 categories are suitable for this user add that as highly recommended product.
                        Your next input will be the info of the person, the name of the product, and screeshot of the ingredients
            '''),
        HumanMessagePromptTemplate.from_template(
            template= 
                '''The person's info is as follows:
                Person info: I am a {gender} who is {age} years old, my weight is {weight}, my height is {height}, I am {diet_type} and I want to {health_goal}. Allergic to {allergen}.
                Product info: {product_info_str}
                'image_url': "data:image/jpeg;base64,{image_data}"'''

            
           
            
            ),
       
    ]
)

In [19]:
chain = chat_prompt | model 

# Path to the image file
image_path = r"C:\Users\DELL\Downloads\13-Google-GenAI-Hack-24\GenAI-Google--Hack-24\Built-RAG\Screenshot 2024-10-02 203723.png"

# Read and convert the image to base64
with open(image_path, "rb") as image_file:
    image_data = base64.b64encode(image_file.read()).decode("utf-8")




In [20]:
print(image_data)

...


In [21]:
response_from_model = chain.invoke(
    {
        
        "image_data": image_data,
        "age":"34",
        "weight": "168kg",
        "gender": "male",
        "height":"5 ft",
        "diet_type": "jain",
        "health_goal": "loose 50 kgs",
        "allergen": "peanuts",
        "product_info":"Pringles Potato Chips Desi Masala Tadka Flavor Pack of 4, Crispy Snack for Game Nights Chips (4 x 102 g)",
        "product_info_str": response_grounding
        

    }
)

In [22]:
response_from_model.content


'I am sorry, I cannot see or analyze any image. I am a text-based chat assistant and thus I do not have the capability to process image data. \n'